# Notes

### Input

Response spreadsheet of the combined student form. Each row represents a student's input. Each column stands for a blank of the form:

> `A`: Timestamp
> `B`: Email Address
> `C`: Gender
>
> `D`: Is the last digit of your UNI even or odd?


#### Group 1 (Course bidding + Preference)

- `E-L`: Bids for *Business Analytics, Cloud Computing, Machine Learning,	Data Analytics,	Optimization,	Stochastic,	Simulation,	Computational Discrete Optimization*

- `M-T`: Rank for the courses (No.1 - No.8)


#### Group 2 (Preference + Course bidding + Timeslot bidding)

- `U-AB`: Rank for the courses (No.1 - No.8)

- `AC-AJ`: Bids for *Business Analytics, Cloud Computing, Machine Learning,	Data Analytics,	Optimization,	Stochastic,	Simulation,	Computational Discrete Optimization*

- `AK-AN`: Bids for time slots (9-11 am, 12-2 pm, 3-5 pm, 6-8 pm)


### Output

`(Student, [3 Courses])` assignment. Each student is assigned to at least one semi-core course.

## Methodologies to test


**Experimental Groups**
* a) Students give a strict ordering of the classes;
* b) Students  bid on classes, so that the total sum of the bidding sums to <= 100;
* c) Students bid on classes from one time slots, so that the total sum of the bidding sums to <= 100;


**Tests**
1. Ignore a, use b to infer students preferences, with class preferences given by higher bidder; [still ask a for comparing with b,c]
2. Use a and b;
3. Use a and c;
4. Ignore b, c, class preferences are given by unique lottery.



## Structure of Notebook
* Data Processing
* Section 1: Preference Generator from Bids and Lotteries
* Section 2: Schedules for Round 2 algorithm
* Section 3: Two-round algorithm 

# Pre Processing
Goal: To separate the raw data file into 2 csv files, one for each experimental group a) and the other for experimental groups b) and c).

**Course Names** 

Let the following courses be denoted by: <br>
`c1`: Business Analytics <br>
`c2`: Cloud Computing <br>
`c3`: Machine Learning <br>
`c4`: Data Analytics <br>
`c5`: Optimization <br>
`c6`: Stochastic <br>
`c7`: Simulation <br>
`c8`: Computational Discrete Optimization 

where `c1`,`c3`,`c5`,`c6` are semi-core

**Time Slots**

Let the following time slots be denoted by: <br>
`t1`: 9-11am (`c6`,`c7`)<br>
`t2`: 12-2pm (`c1`,`c4`)<br>
`t3`: 3-5pm (`c2`,`c8`)<br>
`t4`: 6-8pm (`c3`,`c5`)

**Course Capacities**

Let the course capacities be denoted as:
k (a dict)

## Separate Groups

In [2]:
# authentication
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [4]:
# import StudentForm (Combined) data from Google Sheets
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1LKi4I4VMGEK2_XBi8zYewFSvqx75nb8HFX9Oo08a8qE/edit#gid=1255728861')
data = wb.sheet1.get_all_values()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
filename = "StudentForm (Combined) (Responses).xlsx"

In [16]:
df = pd.read_excel(filename, header=None)
df.shape

(34, 40)

In [17]:
# disagree; this makes the code more compact, and allows both column names 
# and the course names in the preferences to be denoted by the same terms.
# it is necessary for later steps in the code.
df= df.replace(to_replace = {'Business Analytics':'c1', 
                         'Cloud Computing' : 'c2',
                         'Machine Learning' : 'c3',
                         'Data Analytics': 'c4',
                         'Optimization': 'c5',
                         'Stochastic': 'c6',
                         'Simulation': 'c7',
                         'Computational Discrete Optimization': 'c8'
                         },
           value = None)

In [18]:
# df.head()
# df

In [19]:
# extract group 1 and group 2 data from combined df
df_group1 = df[df.iloc[:,3]=="Even (0, 2, 4, 6, 8)"].drop(columns=range(20,40)).replace(to_replace={'Even (0, 2, 4, 6, 8)':'Group1'})
df_group2 = df[df.iloc[:,3]=="Odd (1, 3, 5, 7, 9)"].drop(columns=range(4,20)).replace(to_replace={'Odd (1, 3, 5, 7, 9)':'Group2'})

# set colnames
df_group1_colnames = ['Timestamp','Student','Gender','Group',
                      'c1','c2','c3','c4','c5','c6','c7','c8', # bids on courses
                      'R1','R2','R3','R4','R5','R6','R7','R8'  # ranks
                      ]
df_group2_colnames = ['Timestamp','Student','Gender','Group',
                      'R1','R2','R3','R4','R5','R6','R7','R8', # ranks
                      'c1','c2','c3','c4','c5','c6','c7','c8', # bids on courses
                      't1','t2','t3','t4'                      # bids on time slots
                      ]

df_group1.columns = df_group1_colnames
df_group2.columns = df_group2_colnames

In [20]:
# change datatype of bids from str to int
df_group1 = df_group1.apply(pd.to_numeric, downcast='integer', errors='ignore')
df_group2 = df_group2.apply(pd.to_numeric, downcast='integer', errors='ignore')

# index each student using their UNI
df_group1.index = df_group1.Student.str[:6]
df_group1.index.name = 'UNI'
df_group2.index = df_group2.Student.str[:6]
df_group2.index.name = 'UNI'

# check bid criteria is met
df_group1['CourseBidCriteria'] = (df_group1.loc[:,'c1':'c8'].sum(axis=1) == 100)
df_group2['CourseBidCriteria'] = (df_group2.loc[:,'c1':'c8'].sum(axis=1) == 100)
df_group2['TimeBidCriteria'] = (df_group2.loc[:,'t1':'t4'].sum(axis=1) == 100)

In [21]:
df_group1.head()

,Timestamp,Student,Gender,Group,c1,c2,c3,c4,c5,c6,...,c8,R1,R2,R3,R4,R5,R6,R7,R8,CourseBidCriteria
UNI,,,,,,,,,,,,,,,,,,,,,
qt2131,1596122471600000000,qt2131@columbia.edu,Female,Group1,1,2,3,4,5,6,...,72,c8,c7,c6,c5,c4,c3,c2,c1,True
zs2440,1596215209461000000,zs2440@columbia.edu,Female,Group1,0,100,0,0,0,0,...,0,c2,c1,c3,c4,c5,c6,c7,c8,True
xt2230,1596219805170000000,xt2230@columbia.edu,Female,Group1,0,65,0,0,0,0,...,35,c2,c8,c4,c3,c1,c5,c6,c7,True
sjl222,1596224022427000000,sjl2220@columbia.edu,Male,Group1,45,0,45,0,0,10,...,0,c1,c3,c6,c5,c7,c4,c8,c2,True
zl2856,1596225961121000000,zl2856@columbia.edu,Female,Group1,15,10,15,10,15,15,...,10,c1,c5,c3,c6,c7,c2,c8,c4,True


In [22]:
df_group2.head()

,Timestamp,Student,Gender,Group,R1,R2,R3,R4,R5,R6,...,c5,c6,c7,c8,t1,t2,t3,t4,CourseBidCriteria,TimeBidCriteria
UNI,,,,,,,,,,,,,,,,,,,,,
wr2325,1596123921952000000,wr2325@columbia.edu,Male,Group2,c3,c8,c5,c4,c6,c7,...,10,5,5,35,10,20,40,30,True,True
sj2993,1596215231263000000,sj2993@columbia.edu,Male,Group2,c1,c4,c3,c2,c5,c7,...,3,1,3,3,40,30,20,10,True,True
sa3763,1596215676520000000,sa3763@columbia.edu,Female,Group2,c1,c6,c5,c7,c4,c3,...,5,5,10,5,30,40,20,10,True,True
yf2507,1596215843684000000,yf2507@columbia.edu,Female,Group2,c4,c1,c3,c5,c6,c7,...,5,5,0,0,25,25,25,25,True,True
js5553,1596216166790000000,js5553@columbia.edu,Male,Group2,c1,c3,c2,c4,c5,c6,...,5,5,0,0,0,50,20,30,True,True


## Response Summary

In [23]:
def countBidder(df, course):
    c = df.groupby(course).count()[["Group"]]
    return int(c[1:].sum())

def report(df):
    print(f"# of participants: {len(df)}")
    gender = df.groupby("Gender").count()["Group"]
    print(f"Female: {gender[0]}, Male: {gender[1]}")
    print(f"\n# of bidders for each course:")
    for c in ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']:
        print(f"\t{c}: {countBidder(df, c)}")

In [24]:
report(df_group1)

# of participants: 11
Female: 6, Male: 5

# of bidders for each course:
	c1: 9
	c2: 8
	c3: 6
	c4: 6
	c5: 5
	c6: 7
	c7: 5
	c8: 7


In [25]:
report(df_group2)

# of participants: 22
Female: 10, Male: 12

# of bidders for each course:
	c1: 20
	c2: 16
	c3: 18
	c4: 19
	c5: 19
	c6: 17
	c7: 11
	c8: 7


## Generate Course Capacities

In [105]:
# import math

sc = ['c1','c3','c5','c6']
course = ['c1','c2','c3','c4','c5','c6','c7','c8']
ld = ['c1','c2','c3','c4']
hd = ['c5','c6','c7','c8']

# Equal number of seats for all 8 courses
def capacity(df):
    cap = {c: 0 for c in course}
    
    # 4 semi-core courses take at least len(df) people
    capOfSC = round(len(df)/4)
    lastSC = len(df) - 3*capOfSC
    for c in sc:
        cap[c] += capOfSC
    cap[sc[-1]] = lastSC
    
    remainTotalSeats = 3*len(df) - 3*capOfSC - lastSC
#     print(remainTotalSeats)
    capOfAll = round(remainTotalSeats/8)
    for c in course:
        cap[c] += capOfAll
#     cap[course[-1]] = 3*len(df) - sum(list(cap.values())[:-1])
    
    # adjust for demand
#     for c in ld:
#         cap[c] -= 1
        
#     for c in hd:
#         cap[c] += 1

    return cap

In [235]:
sum(capacity(df_group2).values())
# capacity(df_group1)

70

In [236]:
len(df_group2)

22

In [82]:
[1,2,3][:-1]

[1, 2]

# Section 1: Preference Generator from Bids and Lotteries


### Student-side preferences

```python
def get_pref(df, sc=False):     # actual preferences
def get_bid_pref(df, sc=False): # implied preferences from bids
```
Functions return output of the following structure:
```
{'student1': [c2, c3, ...],
 'student2': [c1, c4, ...],
  ...
}
```

TODO: Function to get dictionary of student preferences from data frame


In [29]:
def get_pref(df, sc=False):
    '''
    Returns a dictionary of students' preferences, with student UNI as the key
    sc=False gives all courses, sc=True gives only semi-core courses
    '''
    pref_dict = {}
    if sc:
        for UNI, row in df.loc[:,'R1':'R8'].iterrows():
            sc_list = []
            for c in row.values:
                if c in ['c1','c3','c5','c6']: # if course is semi-core
                    sc_list.append(c)
            pref_dict[UNI] = sc_list

    else:  
        for UNI, row in df.loc[:,'R1':'R8'].iterrows():
            pref_dict[UNI] = list(row.values)

    return pref_dict

In [52]:
get_pref(df_group2)
# get_pref(df_group2, sc=True)

{'wr2325': ['c3', 'c8', 'c5', 'c4', 'c6', 'c7', 'c2', 'c1'],
 'sj2993': ['c1', 'c4', 'c3', 'c2', 'c5', 'c7', 'c8', 'c6'],
 'sa3763': ['c1', 'c6', 'c5', 'c7', 'c4', 'c3', 'c2', 'c8'],
 'yf2507': ['c4', 'c1', 'c3', 'c5', 'c6', 'c7', 'c8', 'c2'],
 'js5553': ['c1', 'c3', 'c2', 'c4', 'c5', 'c6', 'c7', 'c8'],
 'sc4597': ['c3', 'c1', 'c2', 'c5', 'c6', 'c8', 'c7', 'c4'],
 'pa2561': ['c1', 'c4', 'c5', 'c6', 'c7', 'c2', 'c3', 'c8'],
 'cf2799': ['c3', 'c4', 'c5', 'c1', 'c2', 'c7', 'c6', 'c8'],
 'sc4617': ['c1', 'c5', 'c7', 'c2', 'c3', 'c6', 'c8', 'c4'],
 'ma3973': ['c1', 'c3', 'c5', 'c2', 'c4', 'c6', 'c7', 'c8'],
 'qz2391': ['c4', 'c1', 'c2', 'c3', 'c6', 'c5', 'c7', 'c8'],
 'tnw211': ['c2', 'c1', 'c6', 'c5', 'c7', 'c3', 'c4', 'c8'],
 'xm2235': ['c4', 'c3', 'c6', 'c1', 'c7', 'c5', 'c2', 'c8'],
 'da2899': ['c4', 'c2', 'c3', 'c1', 'c7', 'c8', 'c5', 'c6'],
 'wg2347': ['c4', 'c1', 'c6', 'c8', 'c5', 'c2', 'c3', 'c7'],
 'yp2555': ['c1', 'c4', 'c3', 'c2', 'c5', 'c6', 'c7', 'c8'],
 'yw3379': ['c3', 'c2', 

TODO: Function to generate implied preferences from bids (Test 1).

In [32]:
def get_bid_pref(df, sc=False):
    '''
    Returns a dictionary of students' preferences, derived from course bids.
    Student UNI as key.
    sc=False gives all courses, sc=True gives only semi-core courses
    '''
    pref_dict = {}
    if sc:
        for UNI, row in df.loc[:,'c1':'c8'].iterrows():
            sc_list = []
        for c in row.sort_values(ascending=False).index.values:
            if c in ['c1','c3','c5','c6']: # if course is semi-core
                sc_list.append(c)
        pref_dict[UNI] = sc_list

    else:
        for UNI, row in df.loc[:,'c1':'c8'].iterrows():
            pref_dict[UNI] = list(row.sort_values(ascending=False).index.values)

    return pref_dict

In [53]:
get_bid_pref(df_group1)
# get_bid_pref(df_group1, sc=True)

{'qt2131': ['c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c2', 'c1'],
 'zs2440': ['c2', 'c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c1'],
 'xt2230': ['c2', 'c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c1'],
 'sjl222': ['c3', 'c1', 'c6', 'c8', 'c7', 'c5', 'c4', 'c2'],
 'zl2856': ['c6', 'c5', 'c3', 'c1', 'c8', 'c7', 'c4', 'c2'],
 'mz2776': ['c3', 'c8', 'c6', 'c4', 'c2', 'c1', 'c7', 'c5'],
 'tg2718': ['c4', 'c1', 'c5', 'c3', 'c7', 'c8', 'c6', 'c2'],
 'rg3266': ['c1', 'c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c2'],
 'jy3026': ['c1', 'c3', 'c4', 'c2', 'c8', 'c7', 'c6', 'c5'],
 'atc214': ['c2', 'c1', 'c5', 'c4', 'c8', 'c7', 'c6', 'c3'],
 'wx2226': ['c2', 'c1', 'c7', 'c8', 'c6', 'c5', 'c4', 'c3']}

### Course-side Preferences

```python
def get_course_pref(df, sc = False, rank = True, seed = 42)
# when rank = True (default), returns dictionary of courses' rankings 
# when rank = False, returns dictionary of courses' bids

def get_time_pref(df, sc = False, rank = True, seed = 42)
# converts time bids into course bids, and runs get_course_pref()
```
Function returns a dictionary of the following structure:
```
{'c1': ['student1', 'student3', ...],
 'c2': ['student2', 'student3', ...],
  ...
}
```


TODO: Function to get students' bids for courses/ timeslots (Tests 1-3)

In [34]:
def modified_bid(df, seed = 42):
    '''
    Adds a random real number x drawn from uniform distribution for each student-course pair
    Modifies each positive bid b>0 as b'=b+x
    Returns a modified bid matrix
    '''
    np.random.seed(seed)
    df_ = df.loc[:,'c1':'c8']
    X = np.random.uniform(size=df_.shape)
    mod_bids = df_ + X
    mod_bids[mod_bids < 1] = 0
    return mod_bids

In [35]:
def get_course_pref(df, sc = False, rank = True, seed = 42):
    '''
    Returns course ranks when rank = True, or (original) course bids when rank = False.

    sc = True returns only semi-core courses
    rank = True (default) returns courses' strict ranking on students, based on bids.
          Ties are broken by adds a random number from uniform distribution to actual bid value

    Uses modified_bid():
    seed = 42 is default value; only relevant when rank = True
    '''  
    # for course bids
    if not rank:
        if sc:
            course_dict = df.loc[:,['c1','c3','c5','c6']].to_dict()
        if not sc:
            course_dict = df.loc[:,'c1':'c8'].to_dict()

    # for strict course ranks
    if rank:
        new_df = modified_bid(df, seed = seed)  # get modified bid matrix
        course_dict = {}
        if not sc:       
            for c in new_df:
                course_dict[c] = list(new_df[c].sort_values(ascending=False).index.values)
    if sc:
        for c in ['c1','c3','c5','c6']:
            course_dict[c] = list(new_df[c].sort_values(ascending=False).index.values)

    return course_dict

In [51]:
get_course_pref(df_group1, sc=True)

{'c1': ['rg3266',
  'atc214',
  'sjl222',
  'jy3026',
  'wx2226',
  'tg2718',
  'zl2856',
  'qt2131',
  'mz2776',
  'xt2230',
  'zs2440'],
 'c3': ['mz2776',
  'sjl222',
  'jy3026',
  'zl2856',
  'tg2718',
  'qt2131',
  'wx2226',
  'atc214',
  'rg3266',
  'xt2230',
  'zs2440'],
 'c5': ['zl2856',
  'tg2718',
  'qt2131',
  'atc214',
  'jy3026',
  'wx2226',
  'rg3266',
  'mz2776',
  'sjl222',
  'xt2230',
  'zs2440'],
 'c6': ['zl2856',
  'wx2226',
  'sjl222',
  'qt2131',
  'tg2718',
  'jy3026',
  'mz2776',
  'atc214',
  'rg3266',
  'xt2230',
  'zs2440']}

In [38]:
def get_time_pref(df, sc = False, rank = True, seed = 42):
    '''
    Converts time bids to course bids.
    Returns course ranks when rank = True, or (original) course bids when rank = False.

    Only for df_group2. Returns students' bids timeslots, represented as bids on the individual courses.

    Uses get_course_pref():
      sc = True returns only semi-core courses
      rank = True (default) returns courses' strict ranking on students, based on bids.
          Ties are broken by adds a random number from uniform distribution to actual bid value
      seed = 42 is default value; only relevant when break = True  
    '''
    # check that the dataframe passed into function contains time bids
    try:
        df.loc[:,'t1':'t4']
    except:
        raise ValueError("No timeslot keys in df. This function only works for df_group2.")

    # convert time bids to course bids
    new_df = pd.DataFrame()
    new_df['c1'] = df.t2; new_df['c2'] = df.t3; new_df['c3'] = df.t4; new_df['c4'] = df.t2
    new_df['c5'] = df.t4; new_df['c6'] = df.t1; new_df['c7'] = df.t1; new_df['c8'] = df.t3

    # get_course_pref()
    out_dict = get_course_pref(new_df, sc=sc, rank=rank, seed=seed)

    return out_dict

In [39]:
# get_time_pref(df_group2)

TODO: Function to get course preferences using unique lottery (Test 4)

In [40]:
# np.random.uniform()

In [41]:
def lottery(ds, reverse=False):
    np.random.seed(42)
    ds = ds.reset_index()
    if reverse:
        return np.flip(np.random.permutation(ds['UNI']))
    return np.random.permutation(ds['UNI'])

lottery(df_group1)

# will be good if function can be initialized to give the reverse lottery order as well
# Done!

array(['mz2776', 'qt2131', 'atc214', 'wx2226', 'xt2230', 'zs2440',
       'jy3026', 'zl2856', 'rg3266', 'sjl222', 'tg2718'], dtype=object)

In [42]:
lottery(df_group1, reverse=True)

array(['tg2718', 'sjl222', 'rg3266', 'zl2856', 'jy3026', 'zs2440',
       'xt2230', 'wx2226', 'atc214', 'qt2131', 'mz2776'], dtype=object)

# Section 2: Schedules for the Second Round

TODO: For each student, given their preference list (excluding assigned semi-core course) `pref_r2`, and assigned semi-core course `sc_assigned`, generate list of schedules which satisfy the following constraints:
* All 3 courses do not overlap in course timing
* Includes exactly 2 courses (excluding assigned semi-core course)

```python
def get_schedules(pref_r2, sc_assigned): 
```

# Section 3: Two-round Algorithm 

TODO: Generate course capacities based on the total count of participants.

TODO: Student-proposing algorithms are similar (sc or non-sc). Consider to build a common method for student-proposing.
  ```python
  def propose(students, preference_lists, biddings, number_of_courses_to_be_assigned):
  ```

In [43]:
# student preferences
s_pref_sc = get_bid_pref(df_group1, sc=True)

# course preferences
c_pref_sc = get_course_pref(df_group1, sc=True)

# list of students and courses
student_list = list(s_pref_sc.keys())
course_list = list(c_pref_sc.keys())

In [46]:
## student-proposing GS:

# index preferences to avoid expensive lookups when matching
from collections import defaultdict
srank_sc = defaultdict(dict)
crank_sc = defaultdict(dict)

for s, prefs in s_pref_sc.items():
    for i, c in enumerate(prefs):
        srank_sc[s][c] = i

for c, prefs in c_pref_sc.items():
    for i, s in enumerate(prefs):
        crank_sc[c][s] = i

# functions
def prefers(c,s,h):
    '''Test whether c prefers s over h'''
    return crank_sc[c][s] < crank_sc[c][h]

def after(s,c):
    '''Return the course favored by s after c.'''
    i = srank_sc[s][c] + 1 # index of course following c in list of prefs
    return s_pref_sc[s][i]

In [ ]:
# ^ work in progress, do not delete/edit

### Test 1 (Course bidding only, Group 1)

#### Part I. Assign 1 semi-core to each student

- *Student preferences*: Get implied rankings using 
```python 
get_bid_pref(df_group1, sc = True)
```
- *Course preferences*: Get course bids using 
```python
get_course_pref(df_group1, sc = True)
```
- Carry out student-proposing GS algorithm

- Select each students' biddings of the first sc course on their preference lists as `b_sc_1`;
- For each semi-core course:
  - Order students by bidding `b_sc_1` from high to low
  - Assign top *k* students to this course (*k* = course capacity)
- If there are students who have not been assigned:
  - Excluding students who already have a sc course, select each students' biddings of the second sc course on their preference lists as `b_sc_2`;
  - ...
- Repeat above until all students have 1 sc course.


**Note:** Students assigned during this part will not be rejected later.

In [184]:
sc = ['c1','c3','c5','c6']
course = ['c1','c2','c3','c4','c5','c6','c7','c8']

# get group 1's pref list of sc
pref_sc = get_pref(df_group1, sc=True)

## first round
# get first sc course on each one's pref list
pref_sc_1 = {u: x[0] for (u,x) in pref_sc.items()}

# rank students for each sc by bidding
# (0, uni) means student prefer this sc but placed 0 bid for it
bid_sc = {c: sorted([(df_group1.loc[u,c], u) 
              for u in pref_sc_1.keys() 
              if pref_sc_1[u] == c], reverse=True) 
          for c in sc}

# assign
cap = capacity(df_group1) # init capacity
# keep top k students
bid_sc_k_1 = {c: s[:cap[c]] for (c, s) in bid_sc.items()}
bid_sc_k_1

# # find unmatched student
rejected = [i[1] for l in [s[cap[c]:] for (c, s) in bid_sc.items()] for i in l]
rejected

['zl2856', 'zs2440']

In [187]:
## second round
# get second sc course for the rejected
pref_sc_2 = {u: x[1] for (u,x) in pref_sc.items() if u in rejected}
pref_sc_2

{'zs2440': 'c3', 'zl2856': 'c5'}

In [188]:
# rank students for each sc by bidding
# (0, uni) means student prefer this sc but placed 0 bid for it
bid_sc_2 = {c: sorted([(df_group1.loc[u,c], u) 
              for u in pref_sc_2.keys() 
              if pref_sc_2[u] == c], reverse=True) 
          for c in sc}
bid_sc_2

{'c1': [], 'c3': [(0, 'zs2440')], 'c5': [(15, 'zl2856')], 'c6': []}

In [192]:
# assign
# update capacity
cap_2 = {c: newCap(c, k, bid_sc_2) for (c, k) in cap.items()}
# keep top k students
bid_sc_k_2 = {c: s[:cap_2[c]] for (c, s) in bid_sc_2.items()}
bid_sc_k_2

# find unmatched student
rejected = [i[1] for l in [s[cap_2[c]:] for (c, s) in bid_sc_2.items()] for i in l]
rejected

[]

In [114]:
[1,2,3,4,5,6,7,8,9][:6]

[1, 2, 3, 4, 5, 6]

In [138]:
a = [1,2,3,4,5,6,7,8,9]
a[6:]

[7, 8, 9]

In [158]:
# get group 1's pref list of sc
pref_sc = get_pref(df_group1, sc=True)
len(pref_sc.keys())

11

In [297]:
def newCap(c, k, bid_sc_k):
    a = 0
    if bid_sc_k.get(c):
        a = len(bid_sc_k.get(c))
    return k - a


def assignSC(df):
    stop = False
    r = 0 # starts with 0 (first round)
    cap = capacity(df) # init capacity
    # sc_assign = {c: [] for c in sc}
    pref_sc_l = get_pref(df, sc=True) # get group 1's pref list of sc
    pref_sc_z = {} # pref to be updated each round
    rejected = pref_sc.keys()
    
    while not stop:
        # get first(round#) sc course on each one's pref list
        pref_sc_r = {u: x[r] for (u,x) in pref_sc_l.items() if u in rejected}
        print(pref_sc_r)
        # modify pref list
        pref_sc_z.update(pref_sc_r)

        # rank students for each sc by bidding
        # (0, uni) means student prefer this sc but placed 0 bid for it
        bid_sc = {c: sorted([(df.loc[u,c], u) 
                      for u in pref_sc_z.keys() 
                      if pref_sc_z[u] == c], reverse=True) 
                  for c in sc}

        # keep top k students
        bid_sc_k = {c: s[:cap[c]] for (c, s) in bid_sc.items()}
        print(r, bid_sc_k)

        # find the list of unmatched student unis
        rejected = [i[1] for l in [s[cap[c]:] for (c, s) in bid_sc.items()] for i in l]
        print(rejected)

        if rejected: # not empty
            r += 1
            # cap = {c: newCap(c, k, bid_sc_k) for (c, k) in cap.items()} # update capacity
        else:
            stop = True
        
        # compile assignments
#         for c in bid_sc_k:
#             sc_assign[c].extend(bid_sc_k[c])
    
    # update capacity
    cap = {c: newCap(c, k, bid_sc_k) for (c, k) in cap.items()}
    
    return bid_sc_k, cap

In [298]:
assignCourseView = assignSC(df_group1)
assignCourseView

{'qt2131': 'c6', 'zs2440': 'c1', 'xt2230': 'c3', 'sjl222': 'c1', 'zl2856': 'c1', 'mz2776': 'c3', 'tg2718': 'c1', 'rg3266': 'c1', 'jy3026': 'c1', 'atc214': 'c1', 'wx2226': 'c1'}
0 {'c1': [(100, 'rg3266'), (49, 'atc214'), (45, 'sjl222'), (35, 'jy3026'), (30, 'wx2226'), (20, 'tg2718')], 'c3': [(95, 'mz2776'), (0, 'xt2230')], 'c5': [], 'c6': [(6, 'qt2131')]}
['zl2856', 'zs2440']
{'zs2440': 'c3', 'zl2856': 'c5'}
1 {'c1': [(100, 'rg3266'), (49, 'atc214'), (45, 'sjl222'), (35, 'jy3026'), (30, 'wx2226'), (20, 'tg2718')], 'c3': [(95, 'mz2776'), (0, 'zs2440'), (0, 'xt2230')], 'c5': [(15, 'zl2856')], 'c6': [(6, 'qt2131')]}
[]


({'c1': [(100, 'rg3266'),
   (49, 'atc214'),
   (45, 'sjl222'),
   (35, 'jy3026'),
   (30, 'wx2226'),
   (20, 'tg2718')],
  'c3': [(95, 'mz2776'), (0, 'zs2440'), (0, 'xt2230')],
  'c5': [(15, 'zl2856')],
  'c6': [(6, 'qt2131')]},
 {'c1': 0, 'c2': 3, 'c3': 3, 'c4': 3, 'c5': 5, 'c6': 4, 'c7': 3, 'c8': 3})

In [227]:
a=[]
a.extend([123])
a

[123]

**Part II. Assign 2 courses to each student**

- Remove assigned course, time-conflict course from each student's preference list;

- Students propose to the top 2 courses on their modified preference list;
  -  Each course rejects if outnumbered capacity and holds others in case of rejecting in rounds after;

- Repeat until no one get rejected.

**Note:** The output of above algorithm issemi-core stableand always exits.

In [289]:
# update preference list
prefL = get_pref(df_group1)
updatedPref = prefL
updatedPref

{'qt2131': ['c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c2', 'c1'],
 'zs2440': ['c2', 'c1', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8'],
 'xt2230': ['c2', 'c8', 'c4', 'c3', 'c1', 'c5', 'c6', 'c7'],
 'sjl222': ['c1', 'c3', 'c6', 'c5', 'c7', 'c4', 'c8', 'c2'],
 'zl2856': ['c1', 'c5', 'c3', 'c6', 'c7', 'c2', 'c8', 'c4'],
 'mz2776': ['c3', 'c6', 'c2', 'c1', 'c7', 'c4', 'c8', 'c5'],
 'tg2718': ['c4', 'c1', 'c5', 'c3', 'c7', 'c6', 'c2', 'c8'],
 'rg3266': ['c1', 'c4', 'c3', 'c2', 'c8', 'c5', 'c6', 'c7'],
 'jy3026': ['c1', 'c3', 'c4', 'c2', 'c5', 'c6', 'c7', 'c8'],
 'atc214': ['c1', 'c2', 'c4', 'c5', 'c3', 'c8', 'c6', 'c7'],
 'wx2226': ['c2', 'c1', 'c7', 'c8', 'c6', 'c3', 'c4', 'c5']}

In [269]:
assignCourseView

{'c1': [(100, 'rg3266'),
  (49, 'atc214'),
  (45, 'sjl222'),
  (35, 'jy3026'),
  (30, 'wx2226'),
  (20, 'tg2718')],
 'c3': [(95, 'mz2776'), (0, 'zs2440'), (0, 'xt2230')],
 'c5': [(15, 'zl2856')],
 'c6': [(6, 'qt2131')]}

In [288]:
def courseToStudentView(assignCourseView):
    assignCourseViewUni = {c: [s[1] for s in assignCourseView[c]] for c in assignCourseView.keys()}
    return {u: c for c in assignCourseViewUni.keys() for u in assignCourseViewUni[c]}
assignStudentView = courseToStudentView(assignCourseView)
# assignStudentView

{'rg3266': 'c1',
 'atc214': 'c1',
 'sjl222': 'c1',
 'jy3026': 'c1',
 'wx2226': 'c1',
 'tg2718': 'c1',
 'mz2776': 'c3',
 'zs2440': 'c3',
 'xt2230': 'c3',
 'zl2856': 'c5',
 'qt2131': 'c6'}

In [290]:
# time conflit course pair
coursePair = [['c1', 'c4'], ['c2', 'c8'], ['c3', 'c5'], ['c6', 'c7']]

for u in assignStudentView.keys():
    assignedSC = assignStudentView[u]
    for pair in coursePair:
        if assignedSC in pair:
            updatedPref[u] = [i for i in updatedPref[u] if i not in pair]

updatedPref

{'qt2131': ['c8', 'c5', 'c4', 'c3', 'c2', 'c1'],
 'zs2440': ['c2', 'c1', 'c4', 'c6', 'c7', 'c8'],
 'xt2230': ['c2', 'c8', 'c4', 'c1', 'c6', 'c7'],
 'sjl222': ['c3', 'c6', 'c5', 'c7', 'c8', 'c2'],
 'zl2856': ['c1', 'c6', 'c7', 'c2', 'c8', 'c4'],
 'mz2776': ['c6', 'c2', 'c1', 'c7', 'c4', 'c8'],
 'tg2718': ['c5', 'c3', 'c7', 'c6', 'c2', 'c8'],
 'rg3266': ['c3', 'c2', 'c8', 'c5', 'c6', 'c7'],
 'jy3026': ['c3', 'c2', 'c5', 'c6', 'c7', 'c8'],
 'atc214': ['c2', 'c5', 'c3', 'c8', 'c6', 'c7'],
 'wx2226': ['c2', 'c7', 'c8', 'c6', 'c3', 'c5']}

In [292]:
def courseToStudentView(assignCourseView):
    assignCourseViewUni = {c: [s[1] for s in assignCourseView[c]] for c in assignCourseView.keys()}
    return {u: c for c in assignCourseViewUni.keys() for u in assignCourseViewUni[c]}

def resolveTimeConflict(df, assignCourseView):
    # update preference list
    updatedPref = get_pref(df)
    
    # time conflit course pair
    coursePair = [['c1', 'c4'], ['c2', 'c8'], ['c3', 'c5'], ['c6', 'c7']]
    assignStudentView = courseToStudentView(assignCourseView)
    for u in assignStudentView.keys():
        assignedSC = assignStudentView[u]
        for pair in coursePair:
            if assignedSC in pair:
                updatedPref[u] = [i for i in updatedPref[u] if i not in pair]
                
    return updatedPref

In [299]:
(assignCourseView, cap) = assignSC(df_group1)
updatedPref = resolveTimeConflict(df_group1, assignCourseView)

{'qt2131': 'c6', 'zs2440': 'c1', 'xt2230': 'c3', 'sjl222': 'c1', 'zl2856': 'c1', 'mz2776': 'c3', 'tg2718': 'c1', 'rg3266': 'c1', 'jy3026': 'c1', 'atc214': 'c1', 'wx2226': 'c1'}
0 {'c1': [(100, 'rg3266'), (49, 'atc214'), (45, 'sjl222'), (35, 'jy3026'), (30, 'wx2226'), (20, 'tg2718')], 'c3': [(95, 'mz2776'), (0, 'xt2230')], 'c5': [], 'c6': [(6, 'qt2131')]}
['zl2856', 'zs2440']
{'zs2440': 'c3', 'zl2856': 'c5'}
1 {'c1': [(100, 'rg3266'), (49, 'atc214'), (45, 'sjl222'), (35, 'jy3026'), (30, 'wx2226'), (20, 'tg2718')], 'c3': [(95, 'mz2776'), (0, 'zs2440'), (0, 'xt2230')], 'c5': [(15, 'zl2856')], 'c6': [(6, 'qt2131')]}
[]


In [300]:
assignCourseView

{'c1': [(100, 'rg3266'),
  (49, 'atc214'),
  (45, 'sjl222'),
  (35, 'jy3026'),
  (30, 'wx2226'),
  (20, 'tg2718')],
 'c3': [(95, 'mz2776'), (0, 'zs2440'), (0, 'xt2230')],
 'c5': [(15, 'zl2856')],
 'c6': [(6, 'qt2131')]}

In [301]:
cap

{'c1': 0, 'c2': 3, 'c3': 3, 'c4': 3, 'c5': 5, 'c6': 4, 'c7': 3, 'c8': 3}

In [302]:
updatedPref

{'qt2131': ['c8', 'c5', 'c4', 'c3', 'c2', 'c1'],
 'zs2440': ['c2', 'c1', 'c4', 'c6', 'c7', 'c8'],
 'xt2230': ['c2', 'c8', 'c4', 'c1', 'c6', 'c7'],
 'sjl222': ['c3', 'c6', 'c5', 'c7', 'c8', 'c2'],
 'zl2856': ['c1', 'c6', 'c7', 'c2', 'c8', 'c4'],
 'mz2776': ['c6', 'c2', 'c1', 'c7', 'c4', 'c8'],
 'tg2718': ['c5', 'c3', 'c7', 'c6', 'c2', 'c8'],
 'rg3266': ['c3', 'c2', 'c8', 'c5', 'c6', 'c7'],
 'jy3026': ['c3', 'c2', 'c5', 'c6', 'c7', 'c8'],
 'atc214': ['c2', 'c5', 'c3', 'c8', 'c6', 'c7'],
 'wx2226': ['c2', 'c7', 'c8', 'c6', 'c3', 'c5']}

In [ ]:
# apply to first 2 courses on their prefL
def assign(df, cap, pref):
    stop = False
    r = 0                             # starts with 0 (first round)
#     pref_sc = get_pref(df, sc=True)   # get group 1's pref list of sc
    pref_sc_z = {}                    # pref to be updated each round
    rejected = pref.keys()
    
    while not stop:
        # get first(round#) sc course on each one's pref list
        pref_sc_r = {u: x[r] for (u,x) in pref_sc.items() if u in rejected}
        # modify pref list
        pref_sc_z.update(pref_sc_r)
        # here because later updating bid list would be the same

        # rank students for each sc by bidding
        # (0, uni) means student prefer this sc but placed 0 bid for it
        bid_sc = {c: sorted([(df.loc[u,c], u) 
                      for u in pref_sc_z.keys() 
                      if pref_sc_z[u] == c], reverse=True) 
                  for c in sc}

        # keep top k students
        bid_sc_k = {c: s[:cap[c]] for (c, s) in bid_sc.items()}

        # find the list of unmatched student unis
        rejected = [i[1] for l in [s[cap[c]:] for (c, s) in bid_sc.items()] for i in l]

        if rejected: # not empty
            r += 1
        else:
            stop = True
    
    return bid_sc_k

### Test 2 (Preferences + Course bidding)

#### Part I. Assign 1 semi-core to each student



#### Part II. Assign 2 courses to each student

### Test 3 (Preferences + Timeslot bidding)

#### Part I. Assign 1 semi-core to each student

#### Part II. Assign 2 courses to each student

### Test 4 (Preferences only)

#### Part 0. Reverse lottery to determine course preferences

#### Part I. Assign 1 semi-core to each student

#### Part II. Assign 2 courses to each student